 [View the runnable example on GitHub](https://github.com/intel-analytics/BigDL/tree/main/python/nano/tutorial/notebook/inference/pytorch/accelerate_pytorch_inference_jit_ipex.ipynb)

 # Accelerate PyTorch Inference using jit&ipex

 > 📝 **Note**
> * `jit`: You can use `InferenceOptimizer.trace(..., accelerator="jit")` API to enable the jit acceleration for PyTorch inference.
> * `ipex`: You can use`InferenceOptimizer.trace(...,use_ipex=True)` API to enable the ipex acceleration for PyTorch inference. It only takes a few lines.
> * `jit + ipex`: It is recommended to use JIT and IPEX together. You can user `InferenceOptimizer.trace(..., acclerator="jit", use_ipex=True`) to enable both for PyTorch inference.

 To apply JIT/IPEX acceleration, the following dependencies need to be installed first：

In [ ]:
# for BigDL-Nano
!pip install --pre --upgrade bigdl-nano[pytorch] # install the nightly-bulit version
# !source bigdl-nano-init

 > 📝 **Note**
 >
 > We recommend to run the commands above, especially `source bigdl-nano-init` before jupyter kernel is started, or some of the optimizations may not take effect.

 Let's take an [ResNet-18 model](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html) pretrained on ImageNet dataset as an example. First, we load the model:

In [1]:
import torch
from torchvision.models import resnet18

model_ft = resnet18(pretrained=True)

/home/cpx/anaconda3/envs/junwang-resnext-oob/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/cpx/anaconda3/envs/junwang-resnext-oob/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/home/cpx/anaconda3/envs/junwang-resnext-oob/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-

 Then we set it in evaluation mode:

In [2]:
model_ft.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

 Normal Inference

In [3]:
x = torch.rand(2, 3, 224, 224)
y_hat = model_ft(x)
predictions = y_hat.argmax(dim=1)
print(predictions)

tensor([111, 111])


 Accelerated Inference Using JIT/IPEX/JIT+IPEX, we need import `InferenceOptimizer` firstly.

In [4]:
from bigdl.nano.pytorch import InferenceOptimizer

 JIT Optimizer

In [5]:
jit_model = InferenceOptimizer.trace(model_ft,
                                        accelerator="jit",
                                        input_sample=torch.rand(1, 3, 224, 224))
y_hat = jit_model(x)
predictions = y_hat.argmax(dim=1)
print(predictions)

2022-11-17 13:37:17,867 - torch.distributed.nn.jit.instantiator - INFO - Created a temporary directory at /tmp/tmpdywe5gyl
2022-11-17 13:37:17,868 - torch.distributed.nn.jit.instantiator - INFO - Writing /tmp/tmpdywe5gyl/_remote_module_non_scriptable.py


tensor([111, 111])


 IPEX Optimizer

In [6]:
ipex_model = InferenceOptimizer.trace(model_ft,
                                        use_ipex=True)
y_hat = ipex_model(x)
predictions = y_hat.argmax(dim=1)
print(predictions)

tensor([111, 111])


 IPEX + JIT

In [7]:
jit_ipex_model = InferenceOptimizer.trace(model_ft,
                                            accelerator="jit",
                                            use_ipex=True,
                                            input_sample=torch.rand(1, 3, 224, 224))
y_hat = jit_ipex_model(x)
predictions = y_hat.argmax(dim=1)
print(predictions)

tensor([111, 111])


 > 📝 **Note**
 >
 > `input_sample` is the parameter for OpenVINO accelerator to know the **shape** of the model input. So both the batch size and the specific values are not important to `input_sample`. If we want our test dataset to consist of images with $224 \times 224$ pixels, we could use `torch.rand(1, 3, 224, 224)` for `input_sample` here.
 >
 > Please refer to [API documentation](https://bigdl.readthedocs.io/en/latest/doc/PythonAPI/Nano/pytorch.html#bigdl.nano.pytorch.InferenceOptimizer.trace) for more information on `InferenceOptimizer.trace`.

 > 📚 **Related Readings**
 >
 > - [How to install BigDL-Nano](https://bigdl.readthedocs.io/en/latest/doc/Nano/Overview/nano.html#install)
 > - [How to install BigDL-Nano in Google Colab](https://bigdl.readthedocs.io/en/latest/doc/Nano/Howto/install_in_colab.html)